### As you may have seen we have already completed HR Analytics. It's time to see what is the probability of employee leaving the company. 

### *Import libraries*

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

### *Load dataset*

In [2]:
data = pd.read_csv('../input/hr-analytics/HR_comma_sep.csv')

### *View data*

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


### *Check datatypes*

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction_level       14999 non-null float64
last_evaluation          14999 non-null float64
number_project           14999 non-null int64
average_montly_hours     14999 non-null int64
time_spend_company       14999 non-null int64
Work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
Department               14999 non-null object
salary                   14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


### *Check for null values*

In [5]:
data.isnull().sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
Department               0
salary                   0
dtype: int64

Its great that there no null values. Even if there are it is recommended to remove them. 

### *Converting categorical values to numerical*

As you may have noticed earlier, the salary is categorical and so it has to be converted to numerical. Here, salary is ordinal and so we'll encode it accordingly.  

In [6]:
data.salary = data.salary.astype('category')
data.salary = data.salary.cat.reorder_categories(['low','medium','high'])

data.salary = data.salary.cat.codes

Also, convert department to numerical 

In [7]:
dept = pd.get_dummies(data.Department)

print(dept.head(3))

   IT  RandD  accounting  hr  management  marketing  product_mng  sales  \
0   0      0           0   0           0          0            0      1   
1   0      0           0   0           0          0            0      1   
2   0      0           0   0           0          0            0      1   

   support  technical  
0        0          0  
1        0          0  
2        0          0  


> Dealing with Dummy trap

In [8]:
dept = dept.drop('accounting' , axis=1)

data = data.drop('Department', axis=1)

data = data.join(dept)

data.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,IT,RandD,hr,management,marketing,product_mng,sales,support,technical
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,1,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,0,1,0,1,0,0,0,0,0,0,1,0,0


### Time to begin Prediction Process 

Separating the target column and the features to be considered 

In [9]:
target = data.left
features = data.drop('left', axis=1)

Split the dataset

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
from sklearn.tree import export_graphviz
from sklearn.metrics import recall_score

target_train, target_test, features_train, features_test = train_test_split(target,features,test_size=0.25,random_state=42)

### Decision Tree

In [11]:
model = DecisionTreeClassifier(random_state=42)
model.fit(features_train, target_train)
model.score(features_train,target_train)*100

100.0

In [12]:
model.score(features_test,target_test)*100

97.22666666666666

Let's visualize the graph 

In [13]:
from sklearn import tree
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
import re
export_graphviz(model,"tree.dot")

check_call(['dot','-Tpng','tree.dot','-o','tree.png'])

# Annotating chart with PIL
img = Image.open("tree.png")
draw = ImageDraw.Draw(img)
img.save('sample-out.png')
PImage("sample-out.png", height=2000, width=1900)

Pruning

In [14]:
model_depth_5 = DecisionTreeClassifier(max_depth=5, random_state=42)

# Fit the model
model_depth_5.fit(features_train,target_train)

# Print the accuracy of the prediction for the training set
print(model_depth_5.score(features_train,target_train)*100)

# Print the accuracy of the prediction for the test set
print(model_depth_5.score(features_test,target_test)*100)

97.71535247577563
97.06666666666666


Model Evaluation

In [15]:
 #Predict whether employees will churn using the test set
prediction = model.predict(features_test)

# Calculate precision score by comparing target_test with the prediction
precision_score(target_test, prediction)

0.9240641711229947